In [1]:
   !pip install pillow

  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.0 MB ? eta -:--:--
   --- ------------------------------------ 0.5/7.0 MB 649.3 kB/s eta 0:00:10
   --- ------------------------------------ 0.5/7.0 MB 649.3 kB/s eta 0:00:10
   ---- ----------------------------------- 0.8/7.0 MB 606.8 kB/s eta 0:00:11
   ---- ----------------------------------- 0.8/7.0 MB 606.8 kB/s eta 0:00:11
   ------ --------------------------------- 1.0/7.0 MB 637.7 kB/s eta 0:00:10
   ------- -------------------------------- 1.3/7.0 MB

In [3]:
from PIL import Image
import random

# Dummy function to simulate ML prediction
def predict_fracture(image_path):
    classes = [
        "Avulsion fracture", "Comminuted fracture", "Fracture Dislocation",
        "Greenstick fracture", "Hairline Fracture", "Impacted fracture",
        "Longitudinal fracture", "Oblique fracture", "Pathological fracture",
        "Spiral Fracture", "Not fractured"
    ]
    return random.choice(classes)

# Change this to your image file name
image_path = "xray_sample.jpeg"  # <-- Replace with your actual file name

# Open and show the image
img = Image.open(image_path)
img.show()

# Get prediction
result = predict_fracture(image_path)
print("Prediction:", result)

Prediction: Impacted fracture


In [2]:
   !pip install scikit-learn

  Using cached scikit_learn-1.7.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached numpy-2.3.1-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 2.1 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/8.7 MB 1.9 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.7 MB 1.9 MB/s eta 0:00:04
   -------- ------------------------------- 1.8/8.7 MB 1.9 MB/s eta 0:00:04
   --------- ------------------------------ 2.1/8.7 MB 1.9 MB/s eta 0:00:04
   ------------ --------------------------- 2.6/8.7 MB 1.8 MB/s eta 0:00:04
   ------------- -------------------------- 2.9/8.7 MB 1

In [3]:
   from sklearn.dummy import DummyClassifier
   import joblib

   # Create a dummy model that always predicts "Not fractured"
   model = DummyClassifier(strategy="most_frequent")
   model.fit([[0]], ["Not fractured"])  # Fake training

   # Save the model to the model/ folder
   joblib.dump(model, '../model/model.pkl')
   print("Model saved!")


Model saved!


In [4]:
import os
from PIL import Image
import numpy as np

# Set the path to your train folder (modify if your notebook is elsewhere)
data_dir = '../data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
img_size = (128, 128)  # You can change this size if you want
X = []
y = []

# Loop through each class folder
for label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, label)
    if os.path.isdir(class_dir):
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(class_dir, filename)
                img = Image.open(img_path).convert('L').resize(img_size)
                X.append(np.array(img).flatten())
                y.append(label)

X = np.array(X)
y = np.array(y)
print("Data loaded:", X.shape, y.shape)
print("Classes found:", set(y))

OSError: image file is truncated (1 bytes not processed)

In [5]:
import os
from PIL import Image, ImageFile
import numpy as np

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

data_dir = '../data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
img_size = (128, 128)
X = []
y = []

for label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, label)
    if os.path.isdir(class_dir):
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(class_dir, filename)
                try:
                    img = Image.open(img_path).convert('L').resize(img_size)
                    X.append(np.array(img).flatten())
                    y.append(label)
                except Exception as e:
                    print(f"Skipped {img_path}: {e}")

X = np.array(X)
y = np.array(y)
print("Data loaded:", X.shape, y.shape)
print("Classes found:", set(y))

Data loaded: (9246, 16384) (9246,)
Classes found: {np.str_('not fractured'), np.str_('fractured')}


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(model, '../model/model.pkl')
print("Model saved!")

               precision    recall  f1-score   support

    fractured       1.00      0.99      1.00       922
not fractured       0.99      1.00      1.00       928

     accuracy                           1.00      1850
    macro avg       1.00      1.00      1.00      1850
 weighted avg       1.00      1.00      1.00      1850

Model saved!
